In [1]:
!git clone https://github.com/derInformatiker/AIcrowd-AIBlitz7-Solution.git
!pip install -r AIcrowd-AIBlitz7-Solution/challenge3/requirements.txt
!pip install aicrowd-cli==0.1

Cloning into 'AIcrowd-AIBlitz7-Solution'...
remote: warning: multi-pack bitmap is missing required reverse index
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (313/313), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 313 (delta 184), reused 219 (delta 121), pack-reused 0
Receiving objects: 100% (313/313), 462.25 KiB | 23.11 MiB/s, done.
Resolving deltas: 100% (184/184), done.
     |████████████████████████████████| 10.1MB 17.5MB/s 
     |████████████████████████████████| 28.2MB 111kB/s 
     |████████████████████████████████| 829kB 43.8MB/s 
     |████████████████████████████████| 122kB 54.6MB/s 
     |████████████████████████████████| 829kB 45.8MB/s 
     |████████████████████████████████| 112kB 57.3MB/s 
     |████████████████████████████████| 276kB 63.2MB/s 
     |████████████████████████████████| 952kB 43.8MB/s 
     |████████████████████████████████| 1.3MB 45.9MB/s 
     |████████████████████████████████| 143kB 59.3MB/s 
     |█████

     |████████████████████████████████| 40kB 2.5MB/s 
     |████████████████████████████████| 163kB 19.8MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 204kB 12.6MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
  Created wheel for aicrowd-cli: filename=aicrowd_cli-0.1.0-cp37-none-any.whl size=34929 sha256=6c07ad37a298baeeafd99f1368b7f2e3614e008aa7041ea4c763733417b22fec
  Stored in directory: /root/.cache/pip/wheels/7e/ec/cc/a38391f23ad9bd044c91cd5e6e0a30f2a4265a1eb5adedf9f5
Successfully built aicrowd-cli


###RESTART RUNTIME TO USE NEW PACKAGES

In [1]:
API_KEY = ""  # Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [2]:
!aicrowd dataset download --challenge mars-rotation

!rm -rf data
!mkdir data

!unzip -q train.zip  -d data/train
!unzip -q val.zip -d data/val
!unzip -q test.zip  -d data/test

!mv train.csv data/train.csv
!mv val.csv data/val.csv
!mv sample_submission.csv data/sample_submission.csv

sample_submission.csv: 100% 95.9k/95.9k [00:00<00:00, 669kB/s]
test.zip: 100% 76.5M/76.5M [00:06<00:00, 11.8MB/s]
train.csv: 100% 377k/377k [00:00<00:00, 1.22MB/s]
train.zip: 100% 306M/306M [00:59<00:00, 5.12MB/s]
val.csv: 100% 33.7k/33.7k [00:00<00:00, 563kB/s]
val.zip: 100% 30.6M/30.6M [00:02<00:00, 13.9MB/s]


In [3]:
import shutil

shutil.copy('AIcrowd-AIBlitz7-Solution/challenge3/model.py','model.py')
shutil.copy('AIcrowd-AIBlitz7-Solution/challenge3/dataset.py','dataset.py')

'dataset.py'

In [4]:
import dataset
import albumentations as A
import matplotlib.pyplot as plt
import pandas as pd
import torch

In [ ]:
import model
import albumentations as A
from albumentations.augmentations.transforms import Flip
import pytorch_lightning as pl
from pytorch_lightning import Trainer

img_size = 230

if __name__ == '__main__':
    ckpt = pl.callbacks.model_checkpoint.ModelCheckpoint(monitor = 'val_loss',save_top_k = 1,mode = 'min')
    trainer = Trainer(max_epochs = 100,gpus = 1, callbacks = ckpt, precision=16, amp_level='O1',deterministic=True,fast_dev_run = False)
    
    train_tr = A.Compose([
        A.CenterCrop(img_size,img_size,always_apply=True),
        Flip(p=0.25)
    ])
    
    val_tr = A.Compose([
        A.CenterCrop(img_size,img_size,always_apply=True)
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':48,'train_tr':train_tr,'val_tr':val_tr})
    
    #ckpt = torch.load(path)
    #model.load_state_dict(ckpt['state_dict'])
    trainer.fit(model)
    trainer.test(model)

In [ ]:
import model
import torch
import albumentations as A
from albumentations.augmentations.transforms import Flip, VerticalFlip, HorizontalFlip
import pytorch_lightning as pl
from pytorch_lightning import Trainer
m = 0

tr0 = A.Compose([
        A.CenterCrop(230,230,always_apply=True)
])

tr1 = A.Compose([
    A.CenterCrop(230,230,always_apply=True),
    A.VerticalFlip(p = 1)
])

tr2 = A.Compose([
    A.CenterCrop(230,230,always_apply=True),
    A.HorizontalFlip(p = 1)
])

tr3 = A.Compose([
    A.CenterCrop(230,230,always_apply=True),
    A.HorizontalFlip(p = 1),
    A.VerticalFlip(p = 1)
])

tr_list = [tr0,tr1,tr2,tr3]
concat = torch.tensor([])

for tr in tr_list:
    if __name__ == '__main__':

        path = '' #PATH OF LAST CHECKPOINT ================================
        trainer = Trainer(gpus = 1, precision=16, amp_level='O1',deterministic=True)

        train_tr = A.Compose([
            A.CenterCrop(230,230,always_apply=True)
        ])

        val_tr = A.Compose([
            A.CenterCrop(230,230,always_apply=True)
        ])

        m = model.Classifier({'lr':3e-4,'batch_size':48,'train_tr':train_tr,'val_tr':tr})

        ckpt = torch.load(path)
        m.load_state_dict(ckpt['state_dict'])

        out = trainer.predict(m)
        output = torch.tensor([])

        for i in range(len(out)):
            output = torch.cat((output,torch.tensor(out[i]).argmax(1)))
        concat = torch.cat((concat,output.unsqueeze(0)),0)

In [ ]:
output = torch.round(concat[0])

In [ ]:
import model
model.writeSub(output) 